In [ ]:
import os
import random
import gym
import pylab
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Lambda, Add, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import cv2
import threading
from threading import Thread, Lock
import time
import tensorflow_probability as tfp
from typing import Any, List, Sequence, Tuple
from gym.spaces import Dict, Discrete, Box, Tuple
from parametric_distribution import get_parametric_distribution_for_action_space
from collections import deque

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
#gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_virtual_device_configuration(gpus[0],
#          [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4000)])

tfd = tfp.distributions

class OurModel(tf.keras.Model):
    def __init__(self, input_shape, action_space):
        super(OurModel, self).__init__()
        
        self.flatten = Flatten()
        self.dense_0 = Dense(512, activation='relu')
        self.dense_1 = Dense(action_space)
        self.dense_2 = Dense(1)
        
    def call(self, X_input):
        X_input = self.flatten(X_input)
        X_input = self.dense_0(X_input)
        action_logit = self.dense_1(X_input)
        value = self.dense_2(X_input)
        
        return action_logit, value


def safe_log(x):
  """Computes a safe logarithm which returns 0 if x is zero."""
  return tf.where(
      tf.math.equal(x, 0),
      tf.zeros_like(x),
      tf.math.log(tf.math.maximum(1e-12, x)))


def take_vector_elements(vectors, indices):
    """
    For a batch of vectors, take a single vector component
    out of each vector.
    Args:
      vectors: a [batch x dims] Tensor.
      indices: an int32 Tensor with `batch` entries.
    Returns:
      A Tensor with `batch` entries, one for each vector.
    """
    return tf.gather_nd(vectors, tf.stack([tf.range(tf.shape(vectors)[0]), indices], axis=1))


huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)
sparse_ce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
mse_loss = tf.keras.losses.MeanSquaredError()

parametric_action_distribution = get_parametric_distribution_for_action_space(Discrete(6))

class IMPALA_Agent:
    # IMPALA Main Optimization Algorithm
    def __init__(self, env_name):
        # Initialization Environment and parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.action_size = self.env.action_space.n
        self.EPISODES, self.episode, self.max_average = 2000000, 0, -21.0 # specific for pong
        
        self.memory_size = 25000
        self.memory_1 = []
        self.memory_2 = []
        
        self.batch_size = 2
        self.unroll_length = 101
        
        self.ROWS = 80
        self.COLS = 80
        self.REM_STEP = 4
        self.state_size = (self.COLS, self.ROWS, self.REM_STEP)
        
        self.memory_index_1 = 1
        self.memory_index_2 = 1
        
        self.states_1 = np.zeros((self.unroll_length + 1, *self.state_size), dtype=np.float32)
        self.actions_1 = np.zeros((self.unroll_length + 1), dtype=np.int32)
        self.policies_1 = np.zeros((self.unroll_length + 1, self.action_size), dtype=np.float32)
        self.rewards_1 = np.zeros((self.unroll_length + 1), dtype=np.float32)
        self.dones_1 = np.zeros((self.unroll_length + 1), dtype=np.bool)
        
        self.states_2 = np.zeros((self.unroll_length + 1, *self.state_size), dtype=np.float32)
        self.actions_2 = np.zeros((self.unroll_length + 1), dtype=np.int32)
        self.policies_2 = np.zeros((self.unroll_length + 1, self.action_size), dtype=np.float32)
        self.rewards_2 = np.zeros((self.unroll_length + 1), dtype=np.float32)
        self.dones_2 = np.zeros((self.unroll_length + 1), dtype=np.bool)
        
        self.deq_1 = deque(maxlen=1)
        
        self.states = np.zeros((self.batch_size, self.unroll_length, *self.state_size), dtype=np.float32)
        self.actions = np.zeros((self.batch_size, self.unroll_length), dtype=np.int32)
        self.policies = np.zeros((self.batch_size, self.unroll_length, self.action_size), dtype=np.float32)
        self.rewards = np.zeros((self.batch_size, self.unroll_length), dtype=np.float32)
        self.dones = np.zeros((self.batch_size, self.unroll_length), dtype=np.bool)
        
        self.lock = Lock()
        self.lr = 0.0001

        num_hidden_units = 512
        self.image_memory = np.zeros(self.state_size)
        
        # Instantiate plot memory
        self.scores, self.episodes, self.average = [], [], []

        self.Save_Path = 'Models'
        
        if not os.path.exists(self.Save_Path): os.makedirs(self.Save_Path)
        self.path = '{}_IMPALA_{}'.format(self.env_name, self.lr)
        self.model_name = os.path.join(self.Save_Path, self.path)

        # Create Actor-Critic network model
        self.model = OurModel(input_shape=self.state_size, action_space=self.action_size)
        
        self.optimizer = tf.keras.optimizers.Adam(self.lr)

    def remember(self, state, action, policy, reward, done, thread):
        if thread == 0:
            self.states_1[self.memory_index_1] = state
            self.actions_1[self.memory_index_1] = action
            self.policies_1[self.memory_index_1] = policy
            self.rewards_1[self.memory_index_1] = reward
            self.dones_1[self.memory_index_1] = done

            if self.memory_index_1 == 100:
                for i in range(0, 101):
                    self.states[thread][i] = self.states_1[i]
                    self.actions[thread][i] = self.actions_1[i]
                    self.policies[thread][i] = self.policies_1[i]
                    self.rewards[thread][i] = self.rewards_1[i]
                    self.dones[thread][i] = self.dones_1[i]

                self.update(np.expand_dims(self.states[thread], 0),
                            np.expand_dims(self.actions[thread], 0),
                            np.expand_dims(self.policies[thread], 0),
                            np.expand_dims(self.rewards[thread], 0), 
                            np.expand_dims(self.dones[thread], 0))
                    
                #self.deq.append([self.states[thread], self.actions[thread], self.policies[thread], 
                #                 self.rewards[thread], self.dones[thread]])
                
                self.states_1[0] = self.states_1[self.memory_index_1]
                self.actions_1[0] = self.actions_1[self.memory_index_1]
                self.policies_1[0] = self.policies_1[self.memory_index_1]
                self.rewards_1[0] = self.rewards_1[self.memory_index_1]
                self.dones_1[0] = self.dones_1[self.memory_index_1]
 
                self.memory_index_1 = 1
                
            self.memory_index_1 += 1
        elif thread == 1:
            self.states_2[self.memory_index_2] = state
            self.actions_2[self.memory_index_2] = action
            self.policies_2[self.memory_index_2] = policy
            self.rewards_2[self.memory_index_2] = reward
            self.dones_2[self.memory_index_2] = done

            if self.memory_index_2 == 100:
                for i in range(0, 101):
                    self.states[thread][i] = self.states_2[i]
                    self.actions[thread][i] = self.actions_2[i]
                    self.policies[thread][i] = self.policies_2[i]
                    self.rewards[thread][i] = self.rewards_2[i]
                    self.dones[thread][i] = self.dones_2[i]

                self.update(np.expand_dims(self.states[thread], 0),
                            np.expand_dims(self.actions[thread], 0),
                            np.expand_dims(self.policies[thread], 0),
                            np.expand_dims(self.rewards[thread], 0), 
                            np.expand_dims(self.dones[thread], 0))
                    
                #self.deq.append([self.states[thread], self.actions[thread], self.policies[thread], 
                #                 self.rewards[thread], self.dones[thread]])
                    
                self.states_2[0] = self.states_2[self.memory_index_2]
                self.actions_2[0] = self.actions_2[self.memory_index_2]
                self.policies_2[0] = self.policies_2[self.memory_index_2]
                self.rewards_2[0] = self.rewards_2[self.memory_index_2]
                self.dones_2[0] = self.dones_2[self.memory_index_2]
                    
                self.memory_index_2 = 1

            self.memory_index_2 += 1
            
    def act(self, state):
        #print("state.shape: ", state.shape)
        prediction = self.model(state, training=False)
        dist = tfd.Categorical(logits=prediction[0])
        action = int(dist.sample()[0])
        policy = prediction[0]
        
        return action, policy

    def update(self, states, actions, agent_policies, rewards, dones):
        '''
        states.shape:  (8, 100, 80, 80, 4)
        actions.shape:  (8, 100)
        agent_policies.shape:  (8, 100, 6)
        rewards.shape:  (8, 100)
        dones.shape:  (8, 100)
        '''
        states = tf.transpose(states, perm=[1, 0, 2, 3, 4])
        actions = tf.transpose(actions, perm=[1, 0])
        agent_policies = tf.transpose(agent_policies, perm=[1, 0, 2])
        rewards = tf.transpose(rewards, perm=[1, 0])
        dones = tf.transpose(dones, perm=[1, 0])
        
        batch_size = states.shape[0]
        
        online_variables = self.model.trainable_variables
        with tf.GradientTape() as tape:
            tape.watch(online_variables)
            
            # states.shape:  (8, 100, 80, 80, 4)
            states_folded = tf.reshape(states, [states.shape[0]*states.shape[1], states.shape[2], states.shape[3], states.shape[4]])

            learner_output = self.model(states_folded, training=True)
            learner_policies = tf.reshape(learner_output[0], [states.shape[0], states.shape[1], -1])
            learner_values = tf.reshape(learner_output[1], [states.shape[0], states.shape[1], -1])
            
            agent_logits = tf.nn.softmax(agent_policies[:-1])
            actions = actions[:-1]
            rewards = rewards[1:]
            dones = dones[1:]
        
            learner_logits = tf.nn.softmax(learner_policies[:-1])
            
            learner_values = tf.squeeze(learner_values, axis=2)
            
            bootstrap_value = learner_values[-1]
            learner_values = learner_values[:-1]
            
            discounting = 0.99
            discounts = tf.cast(~dones, tf.float32) * discounting
            
            actions = tf.convert_to_tensor(actions, dtype=tf.int32)
            
            target_action_log_probs = parametric_action_distribution.log_prob(learner_policies[:-1], actions)
            behaviour_action_log_probs = parametric_action_distribution.log_prob(agent_policies[:-1], actions)
            
            lambda_ = 1.0
            
            log_rhos = target_action_log_probs - behaviour_action_log_probs
            
            log_rhos = tf.convert_to_tensor(log_rhos, dtype=tf.float32)
            discounts = tf.convert_to_tensor(discounts, dtype=tf.float32)
            rewards = tf.convert_to_tensor(rewards, dtype=tf.float32)
            values = tf.convert_to_tensor(learner_values, dtype=tf.float32)
            bootstrap_value = tf.convert_to_tensor(bootstrap_value, dtype=tf.float32)
            
            clip_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            clip_pg_rho_threshold = tf.convert_to_tensor(1.0, dtype=tf.float32)
            
            rhos = tf.math.exp(log_rhos)
            
            clipped_rhos = tf.minimum(clip_rho_threshold, rhos, name='clipped_rhos')
            
            cs = tf.minimum(1.0, rhos, name='cs')
            cs *= tf.convert_to_tensor(lambda_, dtype=tf.float32)

            values_t_plus_1 = tf.concat([values[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            deltas = clipped_rhos * (rewards + discounts * values_t_plus_1 - values)
        
            acc = tf.zeros_like(bootstrap_value)
            vs_minus_v_xs = []
            for i in range(int(discounts.shape[0]) - 1, -1, -1):
                discount, c, delta = discounts[i], cs[i], deltas[i]
                acc = delta + discount * c * acc
                vs_minus_v_xs.append(acc)  
            
            vs_minus_v_xs = vs_minus_v_xs[::-1]
            
            vs = tf.add(vs_minus_v_xs, values, name='vs')
            vs_t_plus_1 = tf.concat([vs[1:], tf.expand_dims(bootstrap_value, 0)], axis=0)
            clipped_pg_rhos = tf.minimum(clip_pg_rho_threshold, rhos, name='clipped_pg_rhos')
            
            pg_advantages = (clipped_pg_rhos * (rewards + discounts * vs_t_plus_1 - values))
            
            vs = tf.stop_gradient(vs)
            pg_advantages = tf.stop_gradient(pg_advantages)
            
            actor_loss = -tf.reduce_mean(target_action_log_probs * pg_advantages)
            
            baseline_cost = 0.5
            v_error = values - vs
            critic_loss = baseline_cost * 0.5 * tf.reduce_mean(tf.square(v_error))
            
            total_loss = actor_loss + critic_loss

        grads = tape.gradient(total_loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
    
    def replay(self):
        state_list, action_list, policy_list, reward_list, done_list = [], [], [], [], []
        for i in range(0, 2):
            state, action, policy, reward, done = self.deq.popleft()
            
            state_list.append(state)
            action_list.append(action)
            policy_list.append(policy)
            reward_list.append(reward)
            done_list.append(done)
            
        state_array = np.array(state_list)
        action_array = np.array(action_list)
        policy_array = np.array(policy_list)
        reward_array = np.array(reward_list)
        done_array = np.array(done_list)
            
        #print("state_array.shape: ", state_array.shape)
        #print("action_array.shape: ", action_array.shape)
        #print("policy_array.shape: ", policy_array.shape)
        #print("reward_array.shape: ", reward_array.shape)
        #print("done_array.shape: ", done_array.shape)
        
        self.update(state_array, action_array, policy_array, reward_array, done_array)
           
    def load(self, model_name):
        self.model = load_model(model_name, compile=False)

    def save(self):
        self.model.save(self.model_name)

    pylab.figure(figsize=(18, 9))
    def PlotModel(self, score, episode):
        self.scores.append(score)
        self.episodes.append(episode)
        self.average.append(sum(self.scores[-50:]) / len(self.scores[-50:]))
        if str(episode)[-2:] == "00":# much faster than episode % 100
            pylab.plot(self.episodes, self.scores, 'b')
            pylab.plot(self.episodes, self.average, 'r')
            pylab.ylabel('Score', fontsize=18)
            pylab.xlabel('Steps', fontsize=18)
            try:
                pylab.savefig(self.path + ".png")
            except OSError:
                pass

        return self.average[-1]
    
    def imshow(self, image, rem_step=0):
        #print("image[:,:,rem_step].shape: ", image[:,:,rem_step].shape)
        
        cv2.imshow("pong" + str(rem_step), image[:,:,rem_step])
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()

    def GetImage(self, frame):
        #print("frame.shape: ", frame.shape)
        
        # croping frame to 80x80 size
        frame_cropped = frame[35:195:2, ::2,:]
        if frame_cropped.shape[0] != self.COLS or frame_cropped.shape[1] != self.ROWS:
            # OpenCV resize function 
            frame_cropped = cv2.resize(frame, (self.COLS, self.ROWS), interpolation=cv2.INTER_CUBIC)
        
        # converting to RGB (numpy way)
        frame_rgb = 0.299*frame_cropped[:,:,0] + 0.587*frame_cropped[:,:,1] + 0.114*frame_cropped[:,:,2]
        
        # converting to Gray (OpenCV way)
        #frame_gray = cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2GRAY)     
        #print("frame_gray.shape: ", frame_gray.shape)
        
        frame_rgb[frame_rgb < 100] = 0
        frame_rgb[frame_rgb >= 100] = 255
        # dividing by 255 we expresses value to 0-1 representation
        new_frame = np.array(frame_rgb).astype(np.float32) / 255.0

        # push our data by 1 frame, similar as deq() function work
        self.image_memory = np.roll(self.image_memory, 1, axis=2)

        # inserting new frame to free space
        self.image_memory[:,:,0] = new_frame

        # show image frame   
        #self.imshow(self.image_memory, 0)
        #self.imshow(self.image_memory, 1)
        #self.imshow(self.image_memory, 2)
        #self.imshow(self.image_memory, 3)

        return np.expand_dims(self.image_memory, axis=0)
        
    def reset(self, env):
        frame = env.reset()
        for i in range(self.REM_STEP):
            state = self.GetImage(frame)

        return state

    def step(self, action, env):
        next_state, reward, done, info = env.step(action)
        next_state = self.GetImage(next_state)
        
        return next_state, reward, done, info
    
    def train(self, n_threads):
        self.env.close()
        # Instantiate one environment per thread
        envs = [gym.make(self.env_name) for i in range(n_threads)]

        # Create threads
        threads = [threading.Thread(
                target=self.train_threading,
                daemon=True,
                args=(self, envs[i], i)) for i in range(n_threads)]

        for t in threads:
            time.sleep(2)
            t.start()
            
        for t in threads:
            time.sleep(10)
            t.join()
    
    def render(self, obs):
        cv2.imshow('obs', obs)
        cv2.waitKey(1)
    
    def train_threading(self, agent, env, thread):
        max_average = 15.0
        total_step_1 = 0
        total_step_2 = 0
        for e in range(self.EPISODES):
            state = self.reset(env)

            done = False
            score = 0
            SAVING = ''
            while not done:
                #self.env.render()
                
                action, policy = self.act(state)
                next_state, reward, done, _ = self.step(action, env)
                
                #if self.memory_index != 2:
                self.lock.acquire()
                self.remember(state, action, policy, reward / 20.0, done, thread)
                self.lock.release()

                state = next_state
                score += reward
                if done:
                    break
                
                #if thread == 0:
                    #print("len(self.deq): ", len(self.deq))
                #    if len(self.deq) > 2:
                #        self.replay()
                        
                    #total_step_1 += 1
                
            # Update episode count
            with self.lock:
                average = self.PlotModel(score, self.episode)
                # saving best models
                if average >= self.max_average:
                    self.max_average = average
                    #self.save()
                    SAVING = "SAVING"
                else:
                    SAVING = ""
                
                #print("total_step_1: ", total_step_1)
                #print("total_step_2: ", total_step_2)
                print("episode: {}/{}, thread: {}, score: {}, average: {:.2f} {}".format(self.episode, self.EPISODES, thread, score, average, SAVING))
                if(self.episode < self.EPISODES):
                    self.episode += 1
                 
    def test(self, Actor_name, Critic_name):
        self.load(Actor_name, Critic_name)
        
        for e in range(100):
            state = self.reset(self.env)
            done = False
            
            score = 0
            while not done:
                self.env.render()
                action = np.argmax(self.Actor.predict(state))
                state, reward, done, _ = self.step(action, self.env, state)
                score += reward
                if done:
                    print("episode: {}/{}, score: {}".format(e, self.EPISODES, score))
                    break

        self.env.close()


if __name__ == "__main__":
    env_name = 'Pong-v0'
    agent = IMPALA_Agent(env_name)
    
    #agent.run() # use as IMPALAtotal_step_1
    agent.train(n_threads=2) # use as IMPALA
    #agent.test('Models/Pong-v0_A3C_2.5e-05_Actor.h5', '')

2022-01-18 04:28:29.359002: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-01-18 04:28:30.593707: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-01-18 04:28:30.594406: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-01-18 04:28:30.618510: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-18 04:28:30.618544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: kimbring2-ROG-Strix-GA35DX-G35DX
2022-01-18 04:28:30.618549: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: kimbring2-ROG-Strix-GA35DX-G35DX
2022-01-18 04:28:30.618667: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 

episode: 0/2000000, thread: 0, score: -21.0, average: -21.00 SAVING
episode: 1/2000000, thread: 1, score: -19.0, average: -20.00 SAVING
episode: 2/2000000, thread: 0, score: -21.0, average: -20.33 
episode: 3/2000000, thread: 1, score: -21.0, average: -20.50 
episode: 4/2000000, thread: 0, score: -20.0, average: -20.40 
episode: 5/2000000, thread: 1, score: -21.0, average: -20.50 
episode: 6/2000000, thread: 1, score: -21.0, average: -20.57 
episode: 7/2000000, thread: 0, score: -19.0, average: -20.38 
episode: 8/2000000, thread: 1, score: -20.0, average: -20.33 
episode: 9/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 10/2000000, thread: 1, score: -20.0, average: -20.36 
episode: 11/2000000, thread: 0, score: -20.0, average: -20.33 
episode: 12/2000000, thread: 0, score: -20.0, average: -20.31 
episode: 13/2000000, thread: 1, score: -21.0, average: -20.36 
episode: 14/2000000, thread: 0, score: -21.0, average: -20.40 
episode: 15/2000000, thread: 1, score: -20.0, average